In [1]:
import pandas as pd

In [2]:
game = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/rounds.csv')
table = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/table.csv')

In [3]:

game['Date'] = pd.to_datetime(game['Date'])

In [4]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Tournament        254 non-null    object        
 1   Season            254 non-null    int64         
 2   Round             254 non-null    int64         
 3   Date              254 non-null    datetime64[ns]
 4   StartTime         254 non-null    object        
 5   MatchID           254 non-null    int64         
 6   MatchCustomID     254 non-null    object        
 7   MatchStatus       254 non-null    object        
 8   HomeTeamName      254 non-null    object        
 9   HomeTeamNameCode  254 non-null    object        
 10  HomeTeamID        254 non-null    int64         
 11  AwayTeamName      254 non-null    object        
 12  AwayTeamNameCode  254 non-null    object        
 13  AwayTeamID        254 non-null    int64         
 14  HomeScoreFT       254 non-

In [ ]:
teamdim = game[['TeamID', 'TeamName', 'NameCode', 'TeamColors']].drop_duplicates().reset_index(drop=True)
teamdim.set_index('TeamID')


In [ ]:
tournamentdim = game[[]].drop_duplicates().reset_index(drop=True)
tournamentdim.set_index('HomeTeamName')

In [ ]:
seasondim = game[[]].drop_duplicates().reset_index(drop=True)
seasondim.set_index('AwayTeamName')

In [ ]:
facttable = game.merge(teamdim, on='HomeTeamID')\
                .merge(teamdim, on='AwayTeamID')\
                .merge(tournamentdim, on='AwayTeamID')\
                .merge(seasondim, on='AwayTeamID')\
                [['Round']]
facttable